# Section 7: Graph, Topological Sort, Shortest Path

# Graph

Graph consists of a finite set of Vertices (or Nodes) and a set of Edges which connect a pair of nodes.

__Terminologies:__

- __Vertices:__ are the nodes of the graph.
- __Edges:__ are the lines (or paths) that connect the vertices of the graphs.
- __Unweighted Graph:__ are graphs that do not have any weight associated with any edge.
- __Weighted Graph:__ are graphs that have weight associated with their edges.
- __Undirected Graph:__ are graphs that the edges do not have directions associated with them.
- __Directed Graph:__ are graphs that the edges have directions associated with them.
- __Cyclic Graph:__ a graph which has at least one loop.
- __Acyclic Graph:__ a graph which does not have any loop.
- __Tree:__ is a special case of __Directed Acyclic Graph__.


__Representations:__

- __Adjacency Matrix:__ is a square matrix or you can say it is a 2D array. And the elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.
- __Adjacency List:__ is a collection of unorded list used to represent a graph. Each list describes the set of neighbors of a vertex in the graph.

In [1]:
class Graph:
    def __init__(self, graph={}):
        self.graph = graph
        
    def add_edge(self, vertex, edge):
        if vertex in self.graph:
            self.graph[vertex].append(edge)
        else:
            self.graph[vertex] = [edge]
    
adj_lst = {
    "A":["B", "C"],
    "B":["A", "D", "E"],
    "C":["A", "E"],
    "D":["B", "E", "F"],
    "E":["B", "C", "F"],
    "F":["D", "E"],
    
}

graph = Graph(adj_lst)
print(graph.graph)

{'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'E'], 'D': ['B', 'E', 'F'], 'E': ['B', 'C', 'F'], 'F': ['D', 'E']}


# Breadth First Search

It is an algorithm for traversing graph data structure. It starts at some arbitrary vertex of a graph and explores the neighbor nodes (which are at current level) first, before moving to the next level neighbors.

In [2]:
from collections import deque

def breadth_first_search(graph, start):
    dq = deque()
    visited = set()
    result = []
    
    dq.append(start)
    visited.add(start)
    result.append(start)
    
    while dq:
        curr = dq.popleft()

        for nei in graph[curr]:
            if nei not in visited:
                result.append(nei)
                visited.add(nei)
                dq.append(nei)
    
    return result

print(breadth_first_search(adj_lst, "A"))

['A', 'B', 'C', 'D', 'E', 'F']


# Depth First Search

It is an algorithm for traversing graph data structure. It starts at some arbitrary vertex of a graph and explores as far as possible along each edge before backtracking.

In [3]:
# recursive - TC: (v+e), SC: O(v+e)
def depth_first_search(graph, start):
    def _dfs(node):
        if node in visited:
            return
        
        result.append(node)
        visited.add(node)
        for nei in graph[node]:
            _dfs(nei)
    
    result = []
    visited = set()
    _dfs(start)
    return result


def depth_first_search(graph, node, visited=set(), result=[]):
    if node in visited:
        return
    
    result.append(node)
    visited.add(node)
    
    for nei in graph[node]:
        depth_first_search(graph, nei, visited, result)
        
    return result

print(depth_first_search(adj_lst, "A"))

# interative - TC:, SC:
def depth_first_search(graph, start):
    stack = []
    visited = set()
    result = []
    
    stack.append(start)
    visited.add(start)
    
    while stack:
        curr = stack.pop()
        result.append(curr)
        
        for nei in graph[curr]:
            if nei not in visited:
                stack.append(nei)
                visited.add(nei)
    
    return result

print(depth_first_search(adj_lst, "A"))

['A', 'B', 'D', 'E', 'C', 'F']
['A', 'C', 'E', 'F', 'D', 'B']


# Topological Sort

It sorts given actions in such way that if there is a dependency of one action on another, then the dependent action always comes later than its parent action.

In [4]:
def topological_sort(graph):
    def _dfs(node):
        if node in visited:
            return
        
        for nei in graph[node]:
            _dfs(nei)
                
        visited.add(node)
        result.append(node)
    
    result = []
    visited = set()
    for node in graph:
        _dfs(node)
        
    return result[::-1]

tasks = {
    "A":["C"],
    "B":["C", "D"],
    "C":["E"],
    "D":["F"],
    "E":["F", "H"],
    "F":["G"],
    "G":[],
    "H":[],
}

print(topological_sort(tasks))

['B', 'D', 'A', 'C', 'E', 'H', 'F', 'G']


# Single Source Shortest Path Problem

A single source shortest path problem is about finding a path between a given vertex (called source) to all other vertices in the graph such that the total distance between them (source and destination) is minimum.

__The Problem:__

- Five offices in different cities, the cost between these cities are known. Find the cheapest way from head office to branches in different cities.

__Algorithms:__

- BFS
- Dijkstra's Algorithm
- Bellman Ford

In [5]:
# bfs does not work for weighted graphs
# to find shortest path using bfs, we need to enqueue the path instead of only the node
from collections import deque
def bfs(graph, start, end):
    dq = deque()
    visited = set(start)
    path =[start]

    dq.append([start])

    while dq:
        path = dq.popleft()
        node = path[-1]
        
        if node == end:
            return path
            
        for nei in graph[node]:
            if nei not in visited:
                visited.add(nei)
                new_path = list(path)
                new_path.append(nei)
                dq.append(new_path)
            
                

adj_lst = {
    "A":["B", "C"],
    "B":["A", "D", "E"],
    "C":["A", "E"],
    "D":["B", "E", "F"],
    "E":["B", "C", "F"],
    "F":["D", "E"],    
}

bfs(adj_lst, "A", "F")

['A', 'B', 'D', 'F']

# Dijkstra's Algorithm

It finds the shortest path between some start and end node in positive and negative weighted graphs, if there is no any negative cycle.

In [33]:
from collections import deque

def dijkstra(graph, start):
    dq = deque()
    
    aux = {}
    for key in graph:
        aux[key] = {"cost":float("inf"), "prev": []}
        
    aux[start]["cost"] = 0
    
    dq.append(start)
    
    while dq:
        curr = dq.popleft()
        
        for nei in graph[curr]:
            if aux[curr]["cost"] + graph[curr][nei] < aux[nei]["cost"]:
                aux[nei]["cost"] = aux[curr]["cost"] + graph[curr][nei]
                aux[nei]["prev"] += aux[curr]["prev"]
                aux[nei]["prev"].append(curr)
                dq.append(nei)                    

    return aux
    
    
adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

dijkstra(adj, "A")

{'A': {'cost': 0, 'prev': []},
 'B': {'cost': 2, 'prev': ['A']},
 'C': {'cost': 5, 'prev': ['A']},
 'D': {'cost': 3, 'prev': ['A', 'B']},
 'E': {'cost': 5, 'prev': ['A', 'B']},
 'F': {'cost': 13, 'prev': ['A', 'C']},
 'G': {'cost': 14, 'prev': ['A', 'B', 'E']}}

# All Shortest Paths Algorithm


In [40]:
from collections import deque
def find_all_shortest_path(graph):
    def dijkstra(start):
        distances = {}

        for vertex in graph:
            distances[vertex] = {"cost": float("inf"), "prev": None}

        distances[start]["cost"] = 0
        distances[start]["prev"] = start
        dq = deque()
        dq.append(start)

        while dq:
            curr = dq.popleft()

            for nei in graph[curr]:
                if distances[curr]["cost"] + graph[curr][nei] < distances[nei]["cost"]:
                    distances[nei]["cost"] = distances[curr]["cost"] + graph[curr][nei]
                    distances[nei]["prev"] = curr
                    dq.append(nei)

        return distances


In [41]:
adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

dijkstra(adj, "A")

{'A': {'cost': 0, 'prev': 'A'},
 'B': {'cost': 2, 'prev': 'A'},
 'C': {'cost': 5, 'prev': 'A'},
 'D': {'cost': 3, 'prev': 'B'},
 'E': {'cost': 5, 'prev': 'B'},
 'F': {'cost': 13, 'prev': 'C'},
 'G': {'cost': 14, 'prev': 'E'}}

# Minimum Spanning Tree

A minimum spanning tree (MST) is a subset of edges of connected weighted and undirected graph, which:

- connects all vertices together
- no cycle
- minimum total edge

## Disjoint Set

We use Disjoint Set to find cycles in a weighted undirected graph. This data structure is used in the Kruskal's Algorithm.

In [65]:
class DisjointSet:
    def __init__(self, vertices):
        self.vertices = vertices
        self.parent = {v:v for v in vertices}
        self.rank = dict.fromkeys(vertices, 0)
        
    def find(self, item):
        if self.parent[item] == item:
            return item
        return self.find(self.parent[item])
    
    def union(self, x, y):
        xroot = self.find(x)
        yroot = self.find(y)
        
        if self.rank[xroot] < self.rank[yroot]:
            self.parent[xroot] = yroot
        elif self.rank[xroot] > self.rank[yroot]:
            self.parent[yroot] = xroot
        else:
            self.parent[yroot] = xroot
            self.rank[xroot] += 1
    
ds = DisjointSet(adj)

print(ds.find("A"))

ds.union("A", "B")
ds.union("B", "C")
ds.union("E", "B")

print(ds.parent)
print(ds.rank)
print(ds.find("E"))
        

A
{'A': 'A', 'B': 'A', 'C': 'A', 'D': 'D', 'E': 'A', 'F': 'F', 'G': 'G'}
{'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0}
A


# Kruskal's Algorithm

It is a greedy algorithm to find the mininum spanning tree in a graph in two ways:

- add increasing cost edges at each step
- avoid any cycle at each step

Pratical Problems:

- Landing cables
- TV Network
- Tour Operations
- LAN Networks
- A network of pipes for drinking water or natural gas.
- An electric grid
- Single-link Cluster

In [72]:
def create_edges(graph):
    edges = []
    
    for key in graph:
        for v in graph[key]:
            if (v, key, graph[key][v]) not in edges:
                edges.append((key, v, graph[key][v]))
            
    return edges

def kruskal(edges):
    ds = DisjointSet(adj)
    edges = sorted(edges, key=lambda item: item[2])
    total = 0
    mst = []
    for edge in edges:
        if ds.find(edge[0]) != ds.find(edge[1]):
            total += edge[2]
            mst.append(edge)
            ds.union(edge[0], edge[1])
            
    print(total, mst)
            

adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

edges = create_edges(adj)
kruskal(edges)

26 [('B', 'D', 1), ('A', 'B', 2), ('B', 'E', 3), ('A', 'C', 5), ('F', 'G', 7), ('C', 'F', 8)]


# Prim's Algorithm

It is used to find the minimum spanning tree in a graph. It does not use the Disjoint Set data structure.

Pratical Problems:

- Network for roads and Rail tracks connecting all the cities.
- Irrigation channels and placing microwave towers
- Designing a fiber-optic grid or ICs.
- Traveling Salesman Problem.
- Cluster analysis.
- Pathfinding algorithms used in Al(Artificial Intelligence).

In [135]:
from collections import deque

def prim(graph):
    visited = set()
    aux = {vtx:float("inf") for vtx in graph}
    dq = deque()
    dq.append(list(aux)[0])
    aux[list(aux)[0]] = 0
    
    while dq:
        cur = dq.popleft()

        for nei in graph[cur]:
            if nei not in visited:
                if graph[cur][nei] < aux[nei]:
                    aux[nei] = graph[cur][nei]
                    visited.add(cur)
                    dq.append(nei)
    
    return sum(aux.values())
    

adj = {
    "A":{"B":2, "C":5},
    "B":{"A":2, "C":6, "D":1, "E":3},
    "C":{"A":5, "B":6, "F":8},
    "D":{"B":1, "E":4},
    "E":{"B":3, "D":4, "G":9},
    "F":{"C":8, "G":7},
    "G":{"E":9, "F":7},
}

prim(adj)

26

# Kruskal vs Prim

__Kruskal's Algorithm:__

- Landing cables
- TV Network
- Tour Operations
- LAN Networks
- A network of pipes for drinking water or natural gas.
- An electric grid
- Single-link Cluster

__Prim's Algorithm:__

- Network for roads and Rail tracks connecting all the cities.
- Irrigation channels and placing microwave towers
- Designing a fiber-optic grid or ICs.
- Traveling Salesman Problem.
- Cluster analysis.
- Pathfinding algorithms used in Al(Artificial Intelligence)

# Exercise

Airport problem: https://www.youtube.com/watch?v=qz9tKlF431k

In [134]:
from collections import deque
def find_min_connections(airports, routes, starting_airport):
    def _bfs(node):
        dq = deque()
        dq.append(node)
        visited = set()
        visited.add(node)
        
        while dq:
            cur = dq.popleft()
            
            if node in result:
                result[node].append(cur)
            else:
                result[node] = []
                
            for nei in graph[cur]:
                if nei not in visited:
                    visited.add(nei)
                    dq.append(nei)
    
    graph = {airport:[] for airport in airports}
    
    for src, dst in routes:
        graph[src].append(dst)
        
    visited = set()
    result = {}
    
    for v in graph:
        _bfs(v)
    
    aux = []
    for k in result:
        flag = True
        for v in result:
            if k in result[v]:
                flag = False
                
        if flag:
            aux.append(k)
        
    
    for v in aux:
        graph[starting_airport].append(v)
    
    return aux, result

airports =["BGI","CDG","DEL","DOH","DSM","EWR","EYW","HND",
           "ICN","JFK","LGA","LHR","ORD","SAN","SFO","SIN","TLV","BUD"]

routes = [
    ["DSM","ORD"],
    ["ORD","BGI"],
    ["BGI","LGA"],
    ["SIN","CDG"],
    ["CDG","SIN"],
    ["CDG","BUD"],
    ["DEL","DOH"],
    ["DEL","CDG"],
    ["TLV","DEL"],
    ["EWR","HND"],
    ["HND","ICN"],
    ["HND","JFK"],
    ["ICN","JFK"],
    ["JFK","LGA"],
    ["EYW","LHR"],
    ["LHR","SFO"],
    ["SFO","SAN"],
    ["SFO","DSM"],
    ["SAN","EYW"],
]

find_min_connections(airports, routes, "LGA")

(['EWR', 'TLV'],
 {'BGI': ['LGA'],
  'CDG': ['SIN', 'BUD'],
  'DEL': ['DOH', 'CDG', 'SIN', 'BUD'],
  'DOH': [],
  'DSM': ['ORD', 'BGI', 'LGA'],
  'EWR': ['HND', 'ICN', 'JFK', 'LGA'],
  'EYW': ['LHR', 'SFO', 'SAN', 'DSM', 'ORD', 'BGI', 'LGA'],
  'HND': ['ICN', 'JFK', 'LGA'],
  'ICN': ['JFK', 'LGA'],
  'JFK': ['LGA'],
  'LGA': [],
  'LHR': ['SFO', 'SAN', 'DSM', 'EYW', 'ORD', 'BGI', 'LGA'],
  'ORD': ['BGI', 'LGA'],
  'SAN': ['EYW', 'LHR', 'SFO', 'DSM', 'ORD', 'BGI', 'LGA'],
  'SFO': ['SAN', 'DSM', 'EYW', 'ORD', 'LHR', 'BGI', 'LGA'],
  'SIN': ['CDG', 'BUD'],
  'TLV': ['DEL', 'DOH', 'CDG', 'SIN', 'BUD'],
  'BUD': []})

# Minimal Tree

Given a sorted (increasing order) array with unique integer elements, write an algorithm to create a binary search tree with minimal height.

__Example:__

<div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">sortedArray </span><span class="pun">=</span><span class="pln"> </span><span class="pun">[</span><span class="lit">1</span><span class="pun">,</span><span class="lit">2</span><span class="pun">,</span><span class="lit">3</span><span class="pun">,</span><span class="lit">4</span><span class="pun">,</span><span class="lit">5</span><span class="pun">,</span><span class="lit">6</span><span class="pun">,</span><span class="lit">7</span><span class="pun">,</span><span class="lit">8</span><span class="pun">,</span><span class="lit">9</span><span class="pun">]</span></li><li class="L1"><span class="pln">minimalTree</span><span class="pun">(</span><span class="pln">sortedArray</span><span class="pun">)</span></li><li class="L2"><span class="pln">&nbsp;</span></li><li class="L3"><span class="com">#Output</span></li><li class="L4"><span class="pln">&nbsp;</span></li><li class="L5"><span class="pln">   _5__  </span></li><li class="L6"><span class="pln">  </span><span class="pun">/</span><span class="pln">    \ </span></li><li class="L7"><span class="pln">  </span><span class="lit">3</span><span class="pln">    </span><span class="lit">8</span><span class="pln"> </span></li><li class="L8"><span class="pln"> </span><span class="pun">/</span><span class="pln"> \  </span><span class="pun">/</span><span class="pln"> \</span></li><li class="L9"><span class="pln"> </span><span class="lit">2</span><span class="pln"> </span><span class="lit">4</span><span class="pln">  </span><span class="lit">7</span><span class="pln"> </span><span class="lit">9</span></li><li class="L0"><span class="pun">/</span><span class="pln">    </span><span class="pun">/</span><span class="pln">   </span></li><li class="L1"><span class="lit">1</span><span class="pln">    </span><span class="lit">6</span><span class="pln"> </span></li></ol></pre></div></div>

In [163]:
import math
class BSTNode:
    def __init__(self, data=None, left = None, right= None):
        self.data = data
        self.left = left
        self.right = right
        
    def __str__(self, level=0):
        res = "  " * level + str(self.data) + "\n"
        
        if self.left:
            res += self.left.__str__(level+1)
        if self.right:
            res += self.right.__str__(level+1)
            
        return res

def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return BSTNode(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return BSTNode(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
print(minimalTree(sortedArray))

5
  3
    2
      1
    4
  8
    7
      6
    9



# List of Depths

Given a binary search tree, design an algorithm which creates a linked list of all the nodes at each depth (ie , if you have a tree with depth D, you’ll have D linked lists)

In [180]:
# List of Depth
class LinkedList:
    def __init__(self, val):
        self.val = val
        self.next = None
    
    def add(self, val):
        if self.next == None:
            self.next = LinkedList(val)
        else:
            self.next.add(val)
    def __str__(self):
        return "({val})".format(val = self.val) + str(self.next)

class BinaryTree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
    def __str__(self, level=0):
        res = "  " * level + str(self.val) + "\n"
        
        if self.left:
            res += self.left.__str__(level+1)
        if self.right:
            res += self.right.__str__(level+1)
            
        return res
    
def depth(tree):
    if tree is None:
        return 0
    if tree.left is None and tree.right is None:
        return 1
    
    return 1 + max(depth(tree.left), depth(tree.right))

def treeToLinkedList(tree, custDict = {}, d = None):
    def _preorder(node, level=None):
        if node is None:
            return
        if level is None:
            level = depth(node)
        
        if level in custDict:
            custDict[level].add(node.val)
        elif level not in custDict:
            custDict[level] = LinkedList(node.val)
        
        _preorder(node.left, level-1)
        _preorder(node.right, level-1)
    
    _preorder(tree)
    
    return custDict


def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return BinaryTree(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return BinaryTree(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
root = minimalTree(sortedArray)

aux = treeToLinkedList(root)

for k in aux:
    print(k, aux[k])

print(root)

4 (5)None
3 (3)(8)None
2 (2)(4)(7)(9)None
1 (1)(6)None
5
  3
    2
      1
    4
  8
    7
      6
    9



# Check Balanced - LeetCode 110

Implement a function to check if a binary tree is balanced. For the purposes of this question, a balanced tree is defined to be a tree such that the heights of the two subtrees of any node never differ by more than one.

In [181]:
class Node():
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

def isBalanced(root):
    def _get_height(node):
        if node is None:
            return 0
        
        return 1 + max(_get_height(node.left), _get_height(node.right))
    
    def _get_balance(node):
        if node is None:
            return 0
        
        return _get_height(node.left) - _get_height(node.right)
    
    if _get_balance(root) > 1 or _get_balance(root) < -1:
        return False
    
    return True
    

def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return Node(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return Node(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
root = minimalTree(sortedArray)

isBalanced(root)

True

# Validate BST - LeetCode 98

Implement a function to check if a binary tree is a Binary Search Tree.

In [201]:
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.val = value
        self.left = left
        self.right = right
        
    def __str__(self, level=0):
        res = "  " * level + str(self.val) + "\n"
        
        if self.left:
            res += self.left.__str__(level+1)
        if self.right:
            res += self.right.__str__(level+1)
            
        return res
        
def isValidBST(root):
    def _util(node, minimum=float("-inf"), maximum=float("inf")):
        if node is None:
            return True
        
        left = _util(node.left, minimum, node.val)
        right = _util(node.right, node.val, maximum)
        
        if minimum > node.val or node.val > maximum:
            return False
        
        return left and right
    
    result = _util(root)
    return result


def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return TreeNode(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return TreeNode(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
root = minimalTree(sortedArray)

print(isValidBST(root))
print(root)

root.left.val = 0
print(isValidBST(root))
print(root)

True
5
  3
    2
      1
    4
  8
    7
      6
    9

False
5
  0
    2
      1
    4
  8
    7
      6
    9



# In-order Successor in BST - LeetCode 285

Write an algorithm to find the next node (i.e in-order successor) of given node in a binary search tree. You may assume that each node has a link to its parent.

In [259]:
class Node:
    def __init__(self, key, left=None, right=None):
        self.data = key
        self.left = left
        self.right = right
        
    def __str__(self, level=0):
        res = "  " * level + str(self.data) + "\n"
        
        if self.left:
            res += self.left.__str__(level+1)
        if self.right:
            res += self.right.__str__(level+1)
            
        return res

def insert(node, data):
    if node is None:
        return Node(data)
    else:
        if data <= node.data:
            temp = insert(node.left, data)
            node.left = temp
            temp.parent = node
        else:
            temp = insert(node.right, data)
            node.right = temp
            temp.parent = node
    return node

def find_node(root, data):
    if root is None:
        return 
    
    if root.data == data:
        return root
    elif data < root.data:
        return find_node(root.left, data)
    else:
        return find_node(root.right, data)

def find_min(node):
    curr = node
    while curr is not None:
        if curr.left is None:
            break
        curr = curr.left
    return curr.data

def inOrderSuccessor(root, n):
    current = find_node(root, n)

    if current.right is not None:
        return find_min(current.right)
    
    else:
        parent = root
        successor = None
        
        while parent is not None: 
            if current.data < parent.data:
                successor = parent
                parent = parent.left
            elif current.data > parent.data:
                parent = parent.right
            else:
                break
        
        return successor.data if successor else None
        
def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return Node(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return Node(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
root = minimalTree(sortedArray)

for i in range(1,10):
    print(inOrderSuccessor(root, i))

2
3
4
5
6
7
8
9
None


# Build Order

You are given a list of projects and a list of dependencies (which is a list of pairs of projects, where the second project is dependent on the first project). All of a project's dependencies must be built before the project is. Find a build order that will allow the projects to be built. If there is no valid build order, return an error.

In [303]:
def createGraph(projects, dependencies):
    projectGraph = {}
    for project in projects:
        projectGraph[project] = []
    for pairs in dependencies:
        projectGraph[pairs[0]].extend(pairs[1])
    return projectGraph

project = ['a', 'b', 'c', 'd', 'e', 'f']
dependencies = [('a','d'), ('f','b'), ('b','d'), ('f','a'), ('d','c')]


def findBuildOrder(projects, dependencies):
    def _topological(proj):
        visited.add(proj)
        
        for nei in graph[proj]:
            if nei not in visited:
                _topological(nei)
        
        result.append(proj)
    
    def has_cycle(node):
        if node in visited:
            return True

        visited.add(node)
        
        for nei in graph[node]:
            flag = has_cycle(nei)
            if flag:
                return True
        
        visited.remove(node)
        
        return False
            
    graph = createGraph(projects, dependencies)
    
    visited = set()
    
    for node in graph:
        flag = has_cycle(node)
        if flag:
            return "graph has cycle"
    
    result = []
    visited = set()
    
    for p in graph:
        if p not in visited:
            _topological(p)
        
    
    return result[::-1]
    
findBuildOrder(project, dependencies)

['f', 'e', 'b', 'a', 'd', 'c']

# First Common Ancestor - LeetCode 236

Design an algorithm and write code to find the first common ancestor of two nodes in a binary tree. Avoid storing additional nodes in a data structure. NOTE: This is not necessarily a binary search tree.

In [319]:
class Node:
    def __init__(self, key, left=None, right=None):
        self.data = key
        self.left = left
        self.right = right
        
    def __str__(self, level=0):
        res = "  " * level + str(self.data) + "\n"
        
        if self.left:
            res += self.left.__str__(level+1)
        if self.right:
            res += self.right.__str__(level+1)
            
        return res

def insert(node, data):
    if node is None:
        return Node(data)
    else:
        if data <= node.data:
            temp = insert(node.left, data)
            node.left = temp
            temp.parent = node
        else:
            temp = insert(node.right, data)
            node.right = temp
            temp.parent = node
    return node

def find_node(root, data):
    def dfs(node):
        if node is None:
            return False
        if node.data == data:
            return True
        
        return dfs(node.left) or dfs(node.right)

    return dfs(root)

def find_first_common_ancestor(root, node1, node2):
    def util(node):
        if find_node(node.left, node1) and find_node(node.left, node2):
            return util(node.left)
        elif find_node(node.right, node1) and find_node(node.right, node2):
            return util(node.right)
        else:
            return node
    
    node = util(root)
    return node.data
        
def minimalTree(sortedArray):
    def _insert(arr):
        if len(arr) == 0:
            return None
        if len(arr) == 1:
            return Node(arr[0])
        
        mid = int(len(arr)/2)
        left = _insert(arr[:mid])
        right = _insert(arr[mid+1:])
        
        return Node(arr[mid], left, right)
    
    root = _insert(sortedArray)
    return root


sortedArray = [1,2,3,4,5,6,7,8,9]
root = minimalTree(sortedArray)
find_first_common_ancestor(root,1,4)

3